In [1]:
# pip (în kernelul curent)
import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "openpyxl"])


0

In [1]:
import importlib, super8_indicators, super8_short_backtest
importlib.reload(super8_indicators)
importlib.reload(super8_short_backtest)
import pandas as pd
from super8_short_backtest import Super8ShortBacktester, ShortParams
import numpy as np


# 1) Încarcă datele tale (2h, cu price/high/low/volume; Return/Spread sunt calculate automat dacă lipsesc)
df = pd.read_csv("Simboluri_Binance/AVAXUSDC.csv")
# === Cost per tranzacție (per parte) ===
# Alege unul din profilele de mai jos:

# 1) Binance USDⓈ-M Futures (taker) + slippage mic
fee_pct = 0.00050    # 0.05% comision per parte
slip_pct = 0.00005   # 0.005% slippage per parte (conservator)
# 2) Binance Spot (taker) + BNB (-25%) + slippage mic
# fee_pct = 0.00075  # 0.075% comision per parte
# slip_pct = 0.00005 # 0.005% slippage per parte

per_side_cost_pct = fee_pct + slip_pct                     # în procente (fracție)
per_side_cost_log = -np.log(1.0 - per_side_cost_pct)       # transformare la log-return (valoare > 0)
df["Spread"] = per_side_cost_log                           # cost “per schimbare de poziție”

print(f"Cost per parte: {per_side_cost_pct*100:.4f}% "
      f"(round-trip ~ {2*per_side_cost_pct*100:.4f}%)")


# 2) Parametrii indicatori (1-la-1 cu Pine unde are sens)
indicator_params = dict(
    sEma_Length=426, fEma_Length=58,
    ADX_len=10, ADX_smo=8, th=11.53,
    Sst=0.10, Sinc=0.04, Smax=0.40,
    MACD_options="MACD",   # pune "MAC-Z" ca să folosești varianta MAC-Z
    fastLength=22, slowLength=39, signalLength=13,
    lengthz=14, lengthStdev=16, A=-0.1, B=0.5,        # folosite doar dacă MACD_options="MAC-Z"
    volume_f=0.87, sma_Length=81,
    BB_Length=156, BB_mult=13.18,
    bbMinWidth01=9.3, bbMinWidth02=0.0,
    DClength=76
)

# 3) Parametrii strategiei short (TP/SL ca în Pine, simplificat)
short_params = ShortParams(
    Position="Both",            # sau "Short" dacă vrei să interzici complet long-urile
    TP_options="Both",          # "Normal" | "Donchian" | "Both"
    SL_options="Both",          # "Normal" | "ATR" | "Both"
    tp=3.6, sl=8.0,
    atrPeriodSl=100, multiplierPeriodSl=95.77,
    trailOffset=0.38,            # trailing ignorat în implementarea simplă
    start_time=None             # sau pd.Timestamp("2022-01-01", tz="UTC")
)

engine = Super8ShortBacktester(indicator_params, short_params, use_spread=True, log_return=True)
bt = engine.run(df)
cols = ["high","low","Price","volume","Return","Spread","position","trades","cstrategy","creturn"]
bt_x = bt.copy()
if isinstance(bt_x.index, pd.DatetimeIndex) and bt_x.index.tz is not None:
    # păstrează valorile în UTC, doar scoate informația de timezone
    bt_x.index = bt_x.index.tz_localize(None)

bt_x[cols].to_excel("s8_short_backtest_view.xlsx", index=True)
bt.to_csv("s8_short_backtest_ETHUSDT.csv", index=True)
print("Perf:", bt["cstrategy"].iloc[-1], " | Buy&Hold:", bt["creturn"].iloc[-1],
      " | Trades:", int(bt["trades"].sum()))


Cost per parte: 0.0550% (round-trip ~ 0.1100%)
[BT] df rows=19239 | cols=['high', 'low', 'Price', 'volume', 'Return', 'Spread']
[BT] time: 2021-06-01 02:00:00+00:00 → 2025-10-21 12:00:00+00:00 | dup_time=0
[IND] rows=19239 | cols=['sEMA', 'fEMA', 'EMA_longCond', 'EMA_shortCond', 'adx', 'di_plus', 'di_minus', 'ADX_longCond']... | has_BB=True | has_hist=True | has_histmacz=True
[BT] type(ind)=<class 'pandas.core.frame.DataFrame'> | isNone=False
[BT] ind rows=19239
[BT] base rows=19239 | same_index=True
len(df)= 19239  | len(ind)= 19239  | len(base)= 19239
[BT] rates → EMA_short=0.517, ADX_short=0.534, SAR_short=0.500, MACD_short=0.490, VOL_short=0.436, BB_short01=0.000, entry=0.069
[BT] first TRUE EMA_short: 2021-06-01 16:00:00+00:00
[BT] first TRUE ADX_short: 2021-06-01 08:00:00+00:00
[BT] first TRUE SAR_short: 2021-06-01 04:00:00+00:00
[BT] first TRUE MACD_short: 2021-06-01 08:00:00+00:00
[BT] first TRUE VOL_short: 2021-06-01 02:00:00+00:00
[BT] first TRUE entry: 2021-06-07 14:00:00+00

C:\Users\Stefan\Desktop\Curs Trading\Strategii\super8_short_backtest.py:162: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  entry_sig = entry_cond.shift(1).fillna(False).astype(bool)  # intrăm next-bar


[BT] summary → rows=19239, trades=702, pos_rate=0.098, perf=253.4747, bh=1.1126
Perf: 253.47473963272455  | Buy&Hold: 1.1125976792392156  | Trades: 702
